# Interface to batch download Landsat data for use with pyris
In its current state, the pyris package runs locally on your machine and therefor requires that you download your landsat scenes.  

In the original publication, the user is supposed to downlaod the images from Earth Explorer. I found that the Earth Explorer website to be clunky and difficult to use. Instead this notebook utilizes the python api for Google Earth Engine to download landsat imagery.  

This interface is superior in four ways:
   1. it is automateable
   2. it is interactive
   3. it is explicit 
   4. it allows for the filtering of cloudy images


This interface will allow you to specify a region of interest (roi), select all applicable landsat imagery for that region, and then download the selected imagery.  

**Note**: these images are downloaded as multiband TIFFs with unique but relativelty non-descript file names. Rasterio is used to break apart these tiffs into single band images and rename them to according their landsat product ID with the help of a config file automatically generated based on the user's selection.

In [1]:
import ee
import geemap
import pathlib
import os
import rasterio
import json

# Specify your ROI

- Run the cell below to create the interactive map
- after loading the map, run the cells below to load your ROI in one of two ways:
    - **Draw**: interactively draw your ROI on the map. Can save for later
    - **Load**: Load a predefined roi. Must be in geojson format
- either way you choose, the created ROI wil be a `ee.Geometry.Polygon` instance


In [2]:
Map = geemap.Map(center=(-7.0090,-75.1662), zoom=10, basemap="HYBRID")
Map

Map(center=[-7.009, -75.1662], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [3]:
def ee_poly_as_json(ee_feature, out_path, crs):
    """Save a `ee.Geometry` object as geojson"""
    
    # Save roi to disk
    with open(out_path, 'w') as dst:
        roi_dict = {"type": "Feature",
                    "crs": {"type": "name", 
                            "properties": {"name": crs } 
                            },
                     "geometry": {"type":ee_feature.getInfo()["type"],
                                  "coordinates": ee_feature.getInfo()["coordinates"]
                                 },
                     "properties": {"ID": 1}
                    }

        json.dump(roi_dict, dst)
        
    if pathlib.Path(out_path).exists():
        print(f"Saved roi to {out_path}")
    else:
        print("Something went wrong, file was not saved")

In [4]:
def load_or_draw(gee_map):
    
    # Check if the user wants to upload an roi or draw one themselves
    has_roi = input("Do you have a predefined region of interest (roi)? [y/n]: ")
    
    if has_roi.lower() == "y":
        roi_path = input("Paste the file path for your roi here (GeoJSON): ")
        
        roi_json = json.load(open(roi_path))
        
        roi_gee = ee.Geometry.Polygon(roi_json['geometry']['coordinates'])
        
    else:
        # Use the map widget to get the roi
        print("Use the 'Draw Rectangle' widget on the map above to specify your roi")
        
        # Wait for user to finish drawing the roi
        _draw_roi = input("Type 'done' when you have drawn your roi: ")
        
        # Grab the feature that was drawn last and cast to ee.Geometry object
        last_feature = gee_map.draw_last_feature
        roi_gee = last_feature.geometry()
        
        # Allow user to save the feature as geojson for later use
        save_roi = input("Do you want to save the roi to disk? [y/n]: ")
        
        if save_roi.lower() == 'y':
            export_path = input("Paste the file path for your roi output here: ")
            
            # Get crs from GEE map
            map_crs = gee_map.crs["name"]
        
            ee_poly_as_json(roi_gee, export_path, map_crs)
            
    return roi_gee
    

## Create the `roi` variable

In [5]:
roi = load_or_draw(Map)

Do you have a predefined region of interest (roi)? [y/n]: y
Paste the file path for your roi here (GeoJSON): /Users/avan/FLUD/pyris/data/gee_roi.json


In [6]:
roi.getInfo()

{'type': 'Polygon',
 'coordinates': [[[-75.270615, -7.117884],
   [-75.078297, -7.117884],
   [-75.078297, -6.938254],
   [-75.270615, -6.938254],
   [-75.270615, -7.117884]]]}

## Access Landsat data
- There have been several landsat missions since th 1980s: Landsat 5, 7, 8, & 9
- Google Earth Engine has archived many collections of landsat data according to their mission number and level of product provided
- The dictionary below contains all the GEE links to lansat ImageCollection relevant for pyris as well as the band names that we need


In [7]:
# pyris band requirements are listed in misc.misc.LoadLandsatData()

ls_dict = {
    'LS04':{
        'link':'LANDSAT/LT04/C01/T1',
        'bands':['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
    },
    'LS05':{
        'link':'LANDSAT/LT05/C01/T1',
        'bands':['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
    },
    'LS07':{
        'link':'LANDSAT/LE07/C01/T1',
        'bands':['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
    },
    'LS08':{
        'link':'LANDSAT/LC08/C01/T1',
        'bands':['B2','B3','B4','B5', 'B6', 'B7']
    },
    'LS09':{
        'link':'LANDSAT/LC09/C02/T1',
        'bands':['B2','B3','B4','B5', 'B6', 'B7']
    }
}

## Access and filter landsat data

### Specify your landsat program 5, 7, 8, or 9

In [8]:
landsat = "LS09"

collection_link = ls_dict[landsat]['link']
band_list = ls_dict[landsat]['bands']

### Get the image collection and filter

In [9]:
def get_bands(image, band_list):
    """Filter image to only the desired bands"""
    return image.select(band_list)

In [10]:
# Create image collection
collection = (ee.ImageCollection(collection_link)
                .filterBounds(roi)
                  .filter(ee.Filter.lt("CLOUD_COVER", 10))
             )

# Filter for specific bands
collection = collection.map(lambda x: get_bands(x, band_list))

# Add image collection to map above
Map.addLayer(collection)

# Get metadata of image collection
coll_info = collection.getInfo()

# Check number of images returned
print(f"{len(coll_info['features'])} images in current collection:")
[print(feat['properties']['LANDSAT_PRODUCT_ID']) for feat in coll_info["features"]];

2 images in current collection:
LC09_L1TP_007065_20220108_20220122_02_T1
LC09_L1TP_007065_20220617_20220618_02_T1


## Download images to disk

In [11]:
export_dir = pathlib.Path.home() / 'Downloads' / f"gee_export_{collection_link.replace('/', '_')}"
if not export_dir.exists():
    export_dir.mkdir(parents=True)

geemap.ee_export_image_collection(ee_object=collection, 
                                  out_dir=export_dir,
                                  region=roi
                                  )

Total number of images: 2

Exporting 1/2: LC09_007065_20220108.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/avan/Downloads/gee_export_LANDSAT_LC09_C02_T1/LC09_007065_20220108.tif


Exporting 2/2: LC09_007065_20220617.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/avan/Downloads/gee_export_LANDSAT_LC09_C02_T1/LC09_007065_20220617.tif




## Mimic the Earth Explorer file output structure
- PYRIS expects images to come in this form, so we need to manipulate the GEE exports to fit this

```
├── 
├── [LANDSAT_PRODUCT_ID]
│   ├── [LANDSAT_PRODUCT_ID]_B[BAND_NUMBER].tif
│   └── [LANDSAT_PRODUCT_ID]_B[BAND_NUMBER].tif
├── [LANDSAT_PRODUCT_ID]
│   ├── [LANDSAT_PRODUCT_ID]_B[BAND_NUMBER].tif
│   └── [LANDSAT_PRODUCT_ID]_B[BAND_NUMBER].tif
├── ...
```

- So we need to first get the image IDs from each image, then get the band numbers
- Metadata from GEE is saved in the dictionary `coll_info`

### Create directory for each image

In [12]:
def get_product_ids(coll_info:dict) -> list:
    bucket = []
    
    # Loop through all features in the image collection 
    for feat in coll_info["features"]:
        bucket.append(feat["properties"]["LANDSAT_PRODUCT_ID"])
    return bucket

In [13]:
# Make output directory
rename_dir = pathlib.Path.home() / "Downloads" / f"rename_{collection_link.replace('/', '_')}"
if not rename_dir.exists():
    rename_dir.mkdir(parents=True)
    

In [14]:
# Loop through metadata and make output subdirectories
for p_id in get_product_ids(coll_info):
    path = rename_dir / p_id
    if not path.exists():
        path.mkdir()

### Split each output image and save to the right directory

In [15]:
def split_and_save(in_dir:str, out_dir:str, band_names:list) -> None:
    """Split every landsat image in `in_dir` into its component bands and save them as individual images in outdir"""
    
    # Iterate over all tifs created by GEE
    gee_tif_paths = pathlib.Path(in_dir).iterdir()
    
    for path in gee_tif_paths:
        
        # Get components of the path name
        platform, row_path, acq_date = path.stem.split("_")
        
        # Open the tif and get its metadata
        ds = rasterio.open(path)
        profile = ds.profile
        
#         num_bands = profile["count"]
        
        # Create image for each band
        for i, name in enumerate(band_names):
            
            # Get array
            arr = ds.read(i+1)
            
            # Redefine out_profile
            out_profile = profile.copy()
            out_profile["count"] = 1
            
            # Find matching output subdirectory
            out_sub_dir = pathlib.Path(next(out_dir.glob(f"{platform}*{acq_date}*")))
            
            # Create output name with band number
            out_name = f"{out_sub_dir.name}_{name.split('_')[-1]}.TIF"  # SR product bands are named 'SR_B1'
            
            # Create final out_path
            out_path = out_sub_dir / out_name
            
            # Save image to its corresponding directory
            with rasterio.open(out_path, 'w', **out_profile) as dst:
                dst.write(arr, 1)

In [16]:
split_and_save(in_dir=export_dir, out_dir=rename_dir, band_names = band_list)

Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
